In [96]:
import numpy as np
from multiprocessing import Process, Lock, Value, Array, Manager
import random

In [66]:
def work(proc_id, q):
    # Simple example of work.
    while True:
        q_lock.acquire()
        try:
            index = q.get()
            print("Proces id={}: index={}".format(proc_id, index))
        finally:
            q_lock.release()
            
        if index is None:
            break
       
        locks[index].acquire()
        try:
            print("Proces id={}:  old: resources[{}]={}".format(proc_id, index, resources[index]))
            resources[index] = resources[index] + 1
            print("Proces id={}:  new: resources[{}]={}".format(proc_id, index, resources[index]))
        finally:
            locks[index].release()

In [69]:
# Constants
n_proc = 2
n_cities = 3
resources = Array('i',  [2,3,4])
locks = []
for i in range(3):
    locks.append(Lock())

In [71]:
# Queue creation and filling.
m = Manager()
q = m.Queue()
q_lock = Lock()

for i in range(n_cities):
    q.put(i)
for i in range(n_proc):
    q.put(None)
    
pool_process = []
for i in range(n_proc):
    p = Process(target=work, args=(i,q))
    pool_process.append(p)
    p.start()
    
for p in pool_process:
    p.join()

for e in resources:
    print(e)

Proces id=0: index=0
Proces id=0:  old: resources[0]=3
Proces id=0:  new: resources[0]=4
Proces id=1: index=1
Proces id=1:  old: resources[1]=4
Proces id=0: index=2
Proces id=1:  new: resources[1]=5
Proces id=0:  old: resources[2]=5
Proces id=1: index=None
Proces id=0:  new: resources[2]=6
Proces id=0: index=None
4
5
6


In [89]:
loaded = np.loadtxt('../instances/SolvedTSP/att48_weight.txt', dtype=int)

In [94]:
def init_ant_position(N_CITIES, N_ANTS):
    """Position randomly the ants."""
    solutions = []
    for i in range(N_ANTS):
        solutions.append([random.randint(0, N_CITIES - 1)])
    return solutions

In [220]:
av = set(range(10))

In [221]:
av.remove(5)
av.remove(3)

In [222]:
av

{0, 1, 2, 4, 6, 7, 8, 9}

In [200]:
wei = np.full((10,10), 1)

In [201]:
for i in range(10):
    for j in range(10):
        wei[i,j] = random.randint(0, 9)

In [223]:
wei

array([[5, 2, 0, 2, 9, 9, 7, 3, 0, 0],
       [4, 3, 2, 1, 9, 7, 2, 4, 3, 8],
       [3, 8, 6, 1, 6, 0, 2, 9, 5, 0],
       [3, 7, 3, 2, 4, 1, 5, 1, 2, 2],
       [0, 1, 0, 8, 0, 1, 1, 0, 8, 8],
       [4, 5, 8, 2, 6, 5, 9, 7, 4, 8],
       [1, 2, 8, 1, 4, 8, 2, 1, 0, 6],
       [6, 1, 8, 1, 2, 2, 3, 2, 0, 1],
       [5, 7, 0, 8, 3, 4, 2, 3, 3, 7],
       [4, 6, 8, 6, 1, 8, 4, 8, 9, 6]])

In [224]:
wei[5,:]

array([4, 5, 8, 2, 6, 5, 9, 7, 4, 8])

In [225]:
np.where(wei[5,:] == np.min(wei[5,list(av)]))

(array([0, 8]),)

In [231]:
def init_pheromone_matrix(N_CITIES, weights):
    """Init the pheromone matrix with ne nearest neighbors algorithm."""
    available = set(range(N_CITIES))

    solution = []
    solution.append(random.randint(0, N_CITIES - 1))
    available.remove(solution[-1])

    for i in range(N_CITIES - 1):
        index = solution[-1]
        next_city = int(np.where(weights[index, :] == np.min(weights[index, list(available)]))[0])
        solution.append(next_city)
        available.remove(next_city)
    return solution

In [230]:
N_CITIES = 10
solution = init_pheromone_matrix(N_CITIES, wei)

TypeError: only size-1 arrays can be converted to Python scalars